In [1]:
%load_ext autoreload
%autoreload 3

In [7]:
import pandas as pd
import pudl
import sqlalchemy as sa
from pathlib import Path

In [3]:
pudl_engine = sa.create_engine(pudl.workspace.setup.get_defaults()['pudl_db'])
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine)

### Generate Plant Parts List

In [5]:
# make ppl distinct for Panda
# this was adapted from the RMI repo
# takes as input a non-distinct (includes non true grans) ppl
def get_plant_parts_distinct(plant_parts_eia):
    """Get the EIA plant-parts with only the unique granularities."""
    # We want only the records of the EIA plant-parts that are "true
    # granularies" and those which are not duplicates based on their
    # ownership  so the model doesn't get confused as to which option to
    # pick if there are many records with duplicate data
    plant_parts_eia = plant_parts_eia.assign(
        plant_id_report_year_util_id=lambda x: x.plant_id_report_year
        + "_"
        + x.utility_id_pudl.map(str)
    ).astype({"installation_year": "float"})
    plant_parts_distinct = plant_parts_eia[
        (plant_parts_eia["true_gran"]) & (~plant_parts_eia["ownership_dupe"])
    ]
    return plant_parts_distinct

In [ ]:
plant_parts_eia = pudl_out.plant_parts_eia()

In [ ]:
# a little patch because there was one duplicate record due to a bug in ppl generation
# need to check with CG to see if this got fixed
plant_parts_eia = plant_parts_eia[ ~plant_parts_eia.index.duplicated(keep="first")]
plant_parts_eia_distinct = get_plant_parts_distinct(plant_parts_eia)

In [2]:
# it's not necessary to remove columns any more to save on memory
# but these columns seemed non essential for Panda matching
ppl_cols_to_remove = [
    'appro_part_label',
    'appro_record_id_eia',
    'operational_status',
    'operational_status_pudl',
    'ownership_dupe',
    'retirement_date',
    'planned_retirement_date',
    'true_gran'
]

### Add on utility name

If the latest version of the plant parts list is being used then construction year and installation year should already be included.

In [4]:
# currently df is intended to be the distinct plant parts list
def preprocess(df, pudl_engine):
    # join on utility_name_eia
    eia_util = pd.read_sql("utilities_eia", pudl_engine)
    eia_util = eia_util.set_index('utility_id_eia')['utility_name_eia']
    non_null_df = df[~(df.utility_id_eia.isnull())]
    non_null_df = non_null_df.merge(eia_util, how="left", left_on='utility_id_eia', right_index=True, validate="m:1")
    df_util = pd.concat([non_null_df, df[df.utility_id_eia.isnull()]])
    df = df_util.reindex(df.index)
    
    return df_util

### Separate the plant parts list by year

Currently Panda has a memory issue so inputs are broken out by year

In [1]:
def separate_ppl_by_year(ppl_distinct, output_dir):
    dir_path = Path(output_dir)
    for year in ppl_distinct.report_year.unique():
        (dir_path / f"ferc_eia_{year}").mkdir(parents=True, exist_ok=True)
        df = ppl_distinct[ppl_distinct.report_year == year]
        df.to_csv(f"{output_dir}/ferc_eia_{year}/right.csv")